In [1]:
import faiss
import pickle
from pathlib import Path
import ollama

# Paths
INDEX_DIR = Path("data/faiss")

# Load FAISS index
index = faiss.read_index(str(INDEX_DIR / "law_index.faiss"))

# Load metadata
with open(INDEX_DIR / "metadata.pkl", "rb") as f:
    metadata = pickle.load(f)

print(f"FAISS index and metadata loaded. Total chunks: {len(metadata)}")

FAISS index and metadata loaded. Total chunks: 577


In [2]:
def retrieve(query, top_k=5):
    from sentence_transformers import SentenceTransformer
    embedder = SentenceTransformer("all-mpnet-base-v2")
    
    q_emb = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(q_emb, top_k)
    
    results = [metadata[i]["text"] for i in indices[0]]
    return results
    

In [11]:
def ask_gemma(query, context_chunks):
    # Combine retrieved chunks as context
    context = "\n\n".join(context_chunks)
    
    prompt = f"Answer the following legal question based on the provided context:\n\n{context}\n\nQuestion:\n{query}"
    
    # Send prompt to local Gemma-7B via Ollama
    response = ollama.chat(
        model="gemma3:4b",
        messages=[{"role": "user", "content": prompt}]
    )
    
    # Get the text from the first message in the response
    try:
        return response.messages[0].content
    except AttributeError:
        # fallback if the attribute is named differently
        return str(response)

In [13]:
query = "Explain case id 0791-01"

retrieved_chunks = retrieve(query, top_k=3)
answer = ask_gemma(query, retrieved_chunks)

print("Answer:\n", answer)

Answer:
 model='gemma3:4b' created_at='2025-09-27T08:07:11.011833Z' done=True done_reason='stop' total_duration=79477804750 load_duration=160564291 prompt_eval_count=1996 prompt_eval_duration=15594178416 eval_count=744 eval_duration=63719515125 message=Message(role='assistant', content="Okay, let’s break down the legal issues presented by Case ID 0791-01, based on the provided text excerpts. This case appears to be a complex legal dispute involving a computer search and suppression motions. Here's a detailed explanation, synthesizing the key elements:\n\n**1. Case 1: The Automobile Accident Case (David Elliot, J.)**\n\n* **Procedural Issue:** This case was initially intended to proceed by “submission” (a streamlined process where parties agree on all facts and the judge makes the determination). However, because the parties presented conflicting viewpoints on crucial medical issues (specifically, the plaintiff's psychiatric history), the court determined that a traditional trial was ne